In [1]:
import numpy as np
import pandas as pd
import datetime as dt

from power.ml_ops.data import get_pv_data, clean_pv_data


# Notebook of Marius

## Get and clean data

In [2]:
pv_df = pd.read_csv('../raw_data/1980-2022_pv.csv')
pv_df.head(2)

,Unnamed: 0.1,Unnamed: 0,local_time,electricity,irradiance_direct,irradiance_diffuse,temperature,source
0,0,315532800000,1980-01-01 01:00:00+01:00,0.0,0.0,0.0,-1.296,data/pv_data/1980_pv.csv
1,1,315536400000,1980-01-01 02:00:00+01:00,0.0,0.0,0.0,-1.216,data/pv_data/1980_pv.csv


In [3]:
x = pv_df.loc[0,'local_time']
print('x: ', x)
#dt.datetime.strptime("21 June, 2018", "%d %B, %Y")
dt.datetime.strptime(x, "%Y-%m-%d %H:%M:%S%z")

x:  1980-01-01 01:00:00+01:00


datetime.datetime(1980, 1, 1, 1, 0, tzinfo=datetime.timezone(datetime.timedelta(seconds=3600)))

In [4]:
# load data
pv_df = pd.read_csv('../raw_data/1980-2022_pv.csv')

# drop columns
pv_df.drop(columns=['irradiance_direct','irradiance_diffuse','temperature',
                    'source','Unnamed: 0.1'], inplace=True)

# convert dtypes
pv_df.electricity = pv_df.electricity.astype(float)

#pv_df.local_time    = pd.to_datetime(pv_df.local_time) #
pv_df.local_time    = pv_df.local_time.apply(lambda x:
                                            dt.datetime.strptime(x,
                                            "%Y-%m-%d %H:%M:%S%z"))


pv_df['Unnamed: 0'] = pd.to_datetime(pv_df['Unnamed: 0'], unit='ms').dt.tz_localize('UTC')

# name columns
pv_df.rename(columns={'Unnamed: 0': 'utc_time'}, inplace=True)

# show
pv_df.head(2)

,utc_time,local_time,electricity
0,1980-01-01 00:00:00+00:00,1980-01-01 01:00:00+01:00,0.0
1,1980-01-01 01:00:00+00:00,1980-01-01 02:00:00+01:00,0.0


In [5]:
pv_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 376944 entries, 0 to 376943
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype              
---  ------       --------------   -----              
 0   utc_time     376944 non-null  datetime64[ns, UTC]
 1   local_time   376944 non-null  object             
 2   electricity  376944 non-null  float64            
dtypes: datetime64[ns, UTC](1), float64(1), object(1)
memory usage: 8.6+ MB


In [6]:
pv_df.tail(2)

,utc_time,local_time,electricity
376942,2022-12-31 22:00:00+00:00,2022-12-31 23:00:00+01:00,0.0
376943,2022-12-31 23:00:00+00:00,2023-01-01 00:00:00+01:00,0.0


In [7]:
# unique dates match the df length
print('pv_df number of rows: ', pv_df.shape[0])
print('Number of unique values:')
pv_df.nunique()

pv_df number of rows:  376944
Number of unique values:


utc_time       376944
local_time     376944
electricity       892
dtype: int64

In [8]:
# There are no NA values
pv_df.isna().sum()

utc_time       0
local_time     0
electricity    0
dtype: int64

In [9]:
print('Number of negative electricity values: ', len(pv_df[pv_df.electricity < 0.0]))
print('Number of electricity values above max. capacity (1 kw): ', len( pv_df[pv_df.electricity > 1.0]))

Number of negative electricity values:  0
Number of electricity values above max. capacity (1 kw):  0


In [10]:
def get_pv_data() -> pd.DataFrame:
    """
    Load raw data from local directory
    """
    df = pd.read_csv('../raw_data/1980-2022_pv.csv')
    print('# data loaded')
    return df


In [11]:
def clean_pv_data(pv_df: pd.DataFrame) ->pd.DataFrame:
    """
    Remove unnecessary columns and convert to right dtypes
    """
    # remove unnevessary columns
    pv_df.drop(columns=['irradiance_direct','irradiance_diffuse','temperature',
                    'source','Unnamed: 0.1'], inplace=True)

    # convert dtypes
    pv_df.electricity = pv_df.electricity.astype(float)

    pv_df.local_time    = pv_df.local_time.apply(lambda x:
                                            dt.datetime.strptime(x,
                                            "%Y-%m-%d %H:%M:%S%z")) # pd.to_datetime gives warning

    pv_df['Unnamed: 0'] = pd.to_datetime(pv_df['Unnamed: 0'],
                                         unit='ms').dt.tz_localize('UTC')
    # correct column names
    pv_df.rename(columns={'Unnamed: 0': 'utc_time'}, inplace=True)

    print('# data cleaned')
    return pv_df

In [12]:
df = get_pv_data()
df = clean_pv_data(df)
df.head()

# data loaded
# data cleaned


,utc_time,local_time,electricity
0,1980-01-01 00:00:00+00:00,1980-01-01 01:00:00+01:00,0.0
1,1980-01-01 01:00:00+00:00,1980-01-01 02:00:00+01:00,0.0
2,1980-01-01 02:00:00+00:00,1980-01-01 03:00:00+01:00,0.0
3,1980-01-01 03:00:00+00:00,1980-01-01 04:00:00+01:00,0.0
4,1980-01-01 04:00:00+00:00,1980-01-01 05:00:00+01:00,0.0


## Select years

In [66]:
df = get_pv_data()
df = clean_pv_data(df)
df.head(2)

# data loaded
# data cleaned


,utc_time,local_time,electricity
0,1980-01-01 00:00:00+00:00,1980-01-01 01:00:00+01:00,0.0
1,1980-01-01 01:00:00+00:00,1980-01-01 02:00:00+01:00,0.0


In [67]:
st = 1980
en = 1980

start = "1980-01-01 00:00:00"
end   = "1982-12-31 23:00:00"
part_df = df[df.utc_time.between(start, end)]

print(part_df.head(1))
print(part_df.tail(1))

                   utc_time                 local_time  electricity
0 1980-01-01 00:00:00+00:00  1980-01-01 01:00:00+01:00          0.0
                       utc_time                 local_time  electricity
26303 1982-12-31 23:00:00+00:00  1983-01-01 00:00:00+01:00          0.0


In [85]:
def select_years(df: pd.DataFrame, start=1980, end=1982)-> pd.DataFrame:
    """
    Returns the df between start and end year (inclusive)
    """
    start_point = f"{start}-01-01 00:00:00"
    end_point   = f"{end}-12-31 23:00:00"
    df = df[df.utc_time.between(start_point, end_point)]

    n_years = df['utc_time'].dt.year.nunique()
    print(f"# selected {n_years} years")

    return df


In [86]:
test_df = select_years(df, start=1980, end=1983)


# selected 4 years


In [87]:
test_df.head(2)

,utc_time,local_time,electricity
0,1980-01-01 00:00:00+00:00,1980-01-01 01:00:00+01:00,0.0
1,1980-01-01 01:00:00+00:00,1980-01-01 02:00:00+01:00,0.0


In [88]:
test_df.tail(2)

,utc_time,local_time,electricity
35062,1983-12-31 22:00:00+00:00,1983-12-31 23:00:00+01:00,0.0
35063,1983-12-31 23:00:00+00:00,1984-01-01 00:00:00+01:00,0.0


## Create statitics df

In [178]:
years_df = select_years(df, start=1980, end=1982)

years_df['hour_of_year'] = years_df.utc_time.\
                           apply(lambda x: x.strftime("%m%d%H"))

stats_df = years_df[['hour_of_year', 'electricity']]\
    .groupby(['hour_of_year']).agg(['mean','median','std','skew','min','max'])


# selected 3 years


/var/folders/b1/83z4ln8j1yd088xz1fgkbd2c0000gn/T/ipykernel_10481/589407169.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  years_df['hour_of_year'] = years_df.utc_time.\


In [180]:
stats_df.head(24)

electricity                                         
                    mean median       std      skew    min    max
hour_of_year                                                     
010100          0.000000  0.000  0.000000  0.000000  0.000  0.000
010101          0.000000  0.000  0.000000  0.000000  0.000  0.000
010102          0.000000  0.000  0.000000  0.000000  0.000  0.000
010103          0.000000  0.000  0.000000  0.000000  0.000  0.000
010104          0.000000  0.000  0.000000  0.000000  0.000  0.000
010105          0.000000  0.000  0.000000  0.000000  0.000  0.000
010106          0.000000  0.000  0.000000  0.000000  0.000  0.000
010107          0.000333  0.000  0.000577  1.732051  0.000  0.001
010108          0.021667  0.019  0.004619  1.732051  0.019  0.027
010109          0.075000  0.062  0.025159  1.704387  0.059  0.104
010110          0.115667  0.095  0.042911  1.664570  0.087  0.165
010111          0.150667  0.104  0.107858  1.582523  0.074  0.274
010112          0.188667  0.168  0.125285  0.722109  0.075  0.323
010113          0.132333  0.152  0.081304 -1.024818  0.043  0.202
010114          0.025667  0.035  0.019732 -1.652317  0.003  0.039
010115          0.000000  0.000  0.000000  0.000000  0.000  0.000
010116          0.000000  0.000  0.000000  0.000000  0.000  0.000
010117          0.000000  0.000  0.000000  0.000000  0.000  0.000
010118          0.000000  0.000  0.000000  0.000000  0.000  0.000
010119          0.000000  0.000  0.000000  0.000000  0.000  0.000
010120          0.000000  0.000  0.000000  0.000000  0.000  0.000
010121          0.000000  0.000  0.000000  0.000000  0.000  0.000
010122          0.000000  0.000  0.000000  0.000000  0.000  0.000
010123          0.000000  0.000  0.000000  0.000000  0.000  0.000